In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

import pudl
import pandas as pd
import numpy as np
import sqlalchemy as sa
import pudl.output.eia923 as eia
from pudl.workspace.setup import PudlPaths
pudl_engine = sa.create_engine(PudlPaths().pudl_db)


In [ ]:
api_df = eia.get_fuel_cost_avg_eiaapi(eia.FUEL_COST_CATEGORIES_EIAAPI) # coal, oil, gas

In [ ]:
bulk_df = eia.get_fuel_cost_avg_bulk_elec(pudl_engine=pudl_engine)

In [ ]:
bulk_df.head()

In [ ]:
api_df.drop(columns=['name', 'series_id', 'units']).head()

In [ ]:
bulk_df.shape, api_df.shape

In [ ]:
bulk_df.info()

In [ ]:
api_df.info()

In [ ]:
keep_cols = ['state', 'report_date',
'fuel_cost_per_mmbtu', 'fuel_type_code_pudl', 'fuel_cost_from_eiaapi']

In [ ]:
frc_api = eia.fuel_receipts_costs_eia923(
    pudl_engine=pudl_engine,
    fill=True
)[keep_cols + ['fuel_cost_per_unit']]

In [ ]:
frc_bulk = eia.fuel_receipts_costs_eia923(
    pudl_engine=pudl_engine,
    fill=True
)[keep_cols + ['bulk_agg_fuel_cost_per_mmbtu']]

In [ ]:
frc_api.shape, frc_bulk.shape

In [ ]:
mismatch = frc_api['fuel_cost_from_eiaapi'] ^ frc_bulk['fuel_cost_from_eiaapi']

In [ ]:
mismatch.agg(['sum', 'mean'])

In [ ]:
frc_api['fuel_cost_from_eiaapi'].sum()

In [ ]:
frc_bulk['fuel_cost_from_eiaapi'].sum()

In [ ]:
diff = frc_api['fuel_cost_per_mmbtu'].sub(frc_bulk['fuel_cost_per_mmbtu'])

In [ ]:
diff.ne(0).agg(['sum', 'mean'])

In [ ]:
diff.abs().gt(1e-3).agg(['sum', 'mean'])

In [ ]:
frc_bulk['fuel_cost_from_eiaapi'].agg(['sum', 'mean'])

In [ ]:
diff.abs().replace(np.inf, np.nan).replace(0, np.nan).transform(np.log10).hist(bins=100)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# data is all in same order
frc_api.drop(columns=['fuel_cost_per_mmbtu', 'fuel_cost_per_unit']).eq(
    frc_bulk.drop(columns=['fuel_cost_per_mmbtu', 'bulk_agg_fuel_cost_per_mmbtu'])
).all()

In [ ]:
plt.scatter(frc_api['fuel_cost_per_mmbtu'].replace(0, np.nan).transform(np.log10),
frc_bulk['fuel_cost_per_mmbtu'].replace(0, np.nan).transform(np.log10),
s=1, alpha=0.1)